**Installation and imports**

In [ ]:
!pip install psycopg2

In [ ]:
import os
import requests
import numpy as np
from tensorflow.keras.preprocessing import image
import psycopg2
from psycopg2.extras import DictCursor

**Get PostgreSQL database connection parameters from environment variables**

In [ ]:
conn_params = {
    "dbname": os.getenv('DATABASE_NAME'),
    "user": os.getenv('DATABASE_USER'),
    "password": os.getenv('DATABASE_PASS'),
    "host": os.getenv('DATABASE_HOST'),
    "port": os.getenv('DATABASE_PORT')
}

**Define queries**

In [ ]:
incident_id = os.getenv('INCIDENT_ID')

In [ ]:
select_ids_query = """
SELECT scan_image
FROM fracture_incidents
WHERE id = %s;
"""

update_record_query = f"""
UPDATE fracture_incidents
SET 
    fracture_classification  = %s
WHERE 
    id = %s;
"""

**Query database to get unprocessed incidents**

In [ ]:
image_to_process = ''

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    cursor.execute(select_ids_query, (incident_id,))
    image_to_process = cursor.fetchone()
    
    if image_to_process:
        print("Retrieved image to process is:")
        print(record)
    else:
        print(f"No records found with id {incident_id}.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")

**Get model name from environmental variable and define inference URL**

In [ ]:
deployed_model_name = os.getenv('CLASSIFICATION_MODEL_NAME')
rest_url = os.getenv('MODEL_MESH_URL')
infer_url = f'{rest_url}/v2/models/{deployed_model_name}/infer'

**Retrieve MinIO credentials and initialize client**

In [ ]:
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = os.getenv('AWS_S3_BUCKET')
region = os.getenv('AWS_DEFAULT_REGION')

s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

**Download the incident image**

In [ ]:
folder_path = 'incidents/'
download_path = f'{os.getcwd()}/{image_to_process}'

object_key = f'{folder_path}{image_to_process}'

s3.download_file(bucket_name, object_key, download_path)

**Prepare input image**

In [ ]:
img = image.load_img(image_to_process, target_size=(256,256))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0
data = img_array.tolist()

**Define inference function**

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "x",
                "shape": [1, 256, 256, 3],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data)
    response_dict = response.json()
    return response_dict['outputs']

**Perform inference**

In [ ]:
prediction = rest_request(data)
prediction

**Get the predicten en turn into human readable data**

In [ ]:
class_labels=["Avulsion fracture","Comminuted fracture","Fracture Dislocation","Greenstick fracture",
              "Hairline Fracture","Impacted fracture","Longitudinal fracture","Oblique fracture",
              "Pathological fracture","Spiral Fracture"]

predicted_label = class_labels[np.argmax(prediction[0]['data'])]
print(predicted_label)

**Update database with classification results**

In [ ]:
update_data = (
    predicted_label,
    incident_id
)

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    cursor.execute(update_record_query, update_data)

    conn.commit()
    print(f"Record with ID {incident_id} updated successfully.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")